In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/medical-processed/medical_qa_processed.jsonl


## Prepare the Notebook

In [2]:
!pip install transformers datasets accelerate peft bitsandbytes trl huggingface_hub 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 35.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.9/532.9 kB 35.8 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login
login()

In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

In [5]:
import torch
import random
from datasets import load_dataset, Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model
)
from trl import SFTTrainer
import json
import pandas as pd

2026-01-26 17:47:04.012352: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769449624.186174      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769449624.233530      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769449624.610461      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769449624.610493      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769449624.610496      55 computation_placer.cc:177] computation placer alr

In [6]:
MODEL_NAME = "meta-llama/Llama-3.2-3B-Instruct"
OUTPUT_DIR = "/kaggle/working/llama3.2-3b-ultramedical-lora"
DATASET_PATH = "/kaggle/input/medical-processed/medical_qa_processed.jsonl"

In [7]:
# LoRA configuration

LORA_CONFIG = {
    "r": 16,
    "lora_alpha": 32,
    "lora_dropout": 0.05,
    "bias": "none",
    "task_type": "CAUSAL_LM",
    "target_modules": [
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj"
    ]
}


# Updated Training configuration for 2x T4
TRAINING_CONFIG = {
    "output_dir": OUTPUT_DIR,
    "num_train_epochs": 1,
    "per_device_train_batch_size": 2, 
    "gradient_accumulation_steps": 8,
    "gradient_checkpointing": True,
    "optim": "paged_adamw_32bit", # 32bit is often more stable on T4s with 4-bit loading
    "learning_rate": 2e-4,
    "lr_scheduler_type": "cosine",
    "warmup_ratio": 0.1,
    "logging_steps": 10,
    "save_steps": 2000,
    "save_total_limit": 2,
    "max_grad_norm": 1.0,
    "bf16": True,
    "ddp_find_unused_parameters": False,           
    "report_to": "none",
}

#Quantization config for 4-bit loading
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

## Load and Prepare the Dataset

In [8]:
def load_ultramedical_dataset(dataset_path, max_samples=2000, seed=42):
    """
    Load up to `max_samples` random samples from UltraMedical JSONL.
    Uses reservoir sampling for memory efficiency.
    """

    random.seed(seed)
    reservoir = []
    valid_count = 0

    with open(dataset_path, "r", encoding="utf-8") as f:
        for line_num, line in enumerate(f, start=1):
            try:
                record = json.loads(line)

                if "conversations" not in record or len(record["conversations"]) == 0:
                    continue

                valid_count += 1

                # Fill reservoir
                if len(reservoir) < max_samples:
                    reservoir.append(record)
                else:
                    # Replace elements with decreasing probability
                    j = random.randint(0, valid_count - 1)
                    if j < max_samples:
                        reservoir[j] = record

            except json.JSONDecodeError:
                print(f"⚠️ Skipping invalid JSON at line {line_num}")

    df = pd.DataFrame(reservoir)

    print(f"✅ Loaded {len(df)} random samples (from {valid_count} valid records)")

    return Dataset.from_pandas(df)

In [9]:
def format_conversation(example):
    """
    Convert UltraMedical format to Llama 3.1 chat format

    UltraMedical format:
    [
        {"from": "human", "value": "question"},
        {"from": "gpt", "value": "answer"}
    ]

    Llama 3.1 format:
    <|begin_of_text|><|start_header_id|>user<|end_header_id|>
    question<|eot_id|><|start_header_id|>assistant<|end_header_id|>
    answer<|eot_id|>
    """

    formatted_text = "<|begin_of_text|>"

    for turn in example["conversations"]:
        if turn["from"] == "human":
            role = "user"
        elif turn["from"] == "gpt":
            role = "assistant"
        else:
            continue  # skip unknown roles

        content = turn["value"].strip()

        formatted_text += (
            f"<|start_header_id|>{role}<|end_header_id|>\n\n"
            f"{content}<|eot_id|>"
        )

    return {"text": formatted_text}


In [10]:
# Load dataset
print("Loading dataset...")
dataset = load_ultramedical_dataset(DATASET_PATH)

Loading dataset...
✅ Loaded 2000 random samples (from 409262 valid records)


In [11]:
# Format conversations
print("Formatting conversations...")
formatted_dataset = dataset.map(
    format_conversation,
    remove_columns=dataset.column_names
)

Formatting conversations...


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [12]:
# Split into train/validation (90/10 split)
split_dataset = formatted_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

print(f"Training examples: {len(train_dataset)}")
print(f"Validation examples: {len(eval_dataset)}")

Training examples: 1800
Validation examples: 200


In [13]:
# Preview a formatted example
print("\n" + "="*80)
print("EXAMPLE FORMATTED CONVERSATION:")
print("="*80)
print(train_dataset[0]["text"][:500] + "...")
print("="*80 + "\n")


EXAMPLE FORMATTED CONVERSATION:
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Write a reflection on the complexities of diagnosing mental health disorders.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Diagnosing mental health disorders is a multifaceted and intricate process that demands a nuanced understanding of human psychology, behavior, and the socio-cultural factors that influence mental well-being. The complexity of this task is rooted in several factors that make the diagnosis of mental health d...



## Load Model and Tokenizer

In [14]:
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
tokenizer.model_max_length = 512

Loading tokenizer...


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [15]:
print("Loading model in 4-bit...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

Loading model in 4-bit...


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [17]:
# Prepare model for k-bit training
model = prepare_model_for_kbit_training(model)

## Configure LoRA

In [18]:
print("Configuring LoRA...")
lora_config = LoraConfig(**LORA_CONFIG)

# Add LoRA adapters
model = get_peft_model(model, lora_config)

Configuring LoRA...


In [19]:
# Print trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f"\nTrainable parameters: {trainable_params:,} ({100 * trainable_params / total_params:.2f}%)")
print(f"Total parameters: {total_params:,}\n")


Trainable parameters: 9,175,040 (0.51%)
Total parameters: 1,812,638,720



## Training

In [20]:
print("Setting up trainer...")
training_args = TrainingArguments(**TRAINING_CONFIG)

def formatting_func(example):
    return example["text"]

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    processing_class=tokenizer,
    formatting_func=formatting_func
)

Setting up trainer...


/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:2111: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(


Applying formatting function to train dataset:   0%|          | 0/1800 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/1800 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1800 [00:00<?, ? examples/s]

...

Truncating train dataset:   0%|          | 0/1800 [00:00<?, ? examples/s]

Applying formatting function to eval dataset:   0%|          | 0/200 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/200 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/200 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/200 [00:00<?, ? examples/s]

In [21]:
print("\n" + "="*80)
print("STARTING TRAINING")
print("="*80 + "\n")

trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 128009, 'pad_token_id': 128009}.



STARTING TRAINING



Step,Training Loss
10,1.553400
20,1.395900
30,1.341800
40,1.324000
50,1.290200
60,1.309900
70,1.293300
80,1.241900
90,1.286400
100,1.261100


TrainOutput(global_step=113, training_loss=1.3220192310029426, metrics={'train_runtime': 12053.0111, 'train_samples_per_second': 0.149, 'train_steps_per_second': 0.009, 'total_flos': 1.8356700584116224e+16, 'train_loss': 1.3220192310029426})

## Save Model

In [22]:
print("\nSaving model...")
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print(f"\n✅ Model saved to {OUTPUT_DIR}")
print("\nTraining complete! This is your Model_SFT (baseline for all experiments)")


Saving model...

✅ Model saved to /kaggle/working/llama3.1-8b-ultramedical-lora

Training complete! This is your Model_SFT (baseline for all experiments)


In [22]:
import os
import shutil

OUTPUT_DIR = "/kaggle/working/Model_SFT_3B"

print("\nSaving model...")
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print(f"\n✅ Model saved to {OUTPUT_DIR}")

# ---- ZIP THE MODEL ----
zip_path = "/kaggle/working/Model_SFT_3B.zip"

print("\nZipping model...")
shutil.make_archive(
    base_name=zip_path.replace(".zip", ""),
    format="zip",
    root_dir=OUTPUT_DIR
)

print(f"\n✅ Model zipped at {zip_path}")
print("\n🎉 Training complete! This is your Model_SFT (baseline for all experiments)")



Saving model...

✅ Model saved to /kaggle/working/Model_SFT_3B

Zipping model...

✅ Model zipped at /kaggle/working/Model_SFT_3B.zip

🎉 Training complete! This is your Model_SFT (baseline for all experiments)


## Test the Model

In [ ]:
print("\n" + "="*80)
print("TESTING MODEL")
print("="*80 + "\n")

# Merge LoRA weights for inference (optional, for faster inference)
model = model.merge_and_unload()

# Create a text generation pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    temperature=0.7,
    top_p=0.9,
)

# Test prompt
test_prompt = """<|begin_of_text|><|start_header_id|>user<|end_header_id|>

A 55-year-old man presents with chest pain and shortness of breath. What are the key differential diagnoses to consider?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""

print("Test prompt:", test_prompt)
print("\nModel response:")
response = pipe(test_prompt)
print(response[0]["generated_text"])


TESTING MODEL



/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
Device set to use cuda:0
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
Caching is incompatible with gradient checkpointing in LlamaDecoderLayer. Setting `past_key_values=None`.


Test prompt: <|begin_of_text|><|start_header_id|>user<|end_header_id|>

A 55-year-old man presents with chest pain and shortness of breath. What are the key differential diagnoses to consider?<|eot_id|><|start_header_id|>assistant<|end_header_id|>



Model response:
